In [ ]:
from google.colab import drive

drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
cd gdrive

/content/gdrive


In [ ]:
cd MyDrive/

/content/gdrive/MyDrive


In [ ]:
cd Hackathon/

[Errno 2] No such file or directory: 'Hackathon/'
/content/gdrive/MyDrive


In [ ]:
ls

'58072 - Copy.jpg'
'birthday video '/
'birthday wish video.mp4'
'Colab Notebooks'/
'Course structure.gdoc'
 download.jpg
'email1091794979050657227 (1).TXT.gdoc'
'email1091794979050657227 (2).TXT.gdoc'
 email1091794979050657227.TXT.gdoc
 email2434154931484438156.TXT.gdoc
 email3171897430923158520.TXT.gdoc
'email5286399657135139678 (1).TXT.gdoc'
 email5286399657135139678.TXT.gdoc
 email5583467381972692744.TXT.gdoc
 eq_100.png
 eq_101.png
 eq_102.png
 eq_103.png
 eq_104.png
 eq_105.png
 eq_106.png
 eq_107.png
 eq_108.png
 eq_109.png
 eq_110.png
 eq_111.png
 eq_112.png
 eq_113.png
 eq_114.png
 eq_115.png
 eq_116.png
 eq_117.png
 eq_118.png
 eq_119.png
 eq_120.png
 eq_121.png
 eq_122.png
 eq_123.png
 eq_124.png
 eq_125.png
 eq_126.png
 eq_127.png
 eq_128.png
 eq_129.png
 eq_130.png
 eq_131.png
 eq_132.png
 eq_133.png
 eq_134.png
 eq_135.png
 eq_136.png
 eq_137.png
 eq_138.png
 eq_139.png
 eq_140.png
 eq_141.png
 eq_142.png
 eq_143.png
 eq_144.png
 eq_145.png
 eq_146.png
 eq_147.png
 eq_148.

In [ ]:
cd Hackathon\ 

/content/gdrive/My Drive/Hackathon 


In [ ]:
import pandas as pd
import numpy as np 

train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [ ]:
train.fillna(0,inplace=True)
test.fillna(0,inplace=True)


In [ ]:
train['Credit_Product'] = train['Credit_Product'].replace(['Yes','No'],[1,0])
test['Credit_Product'] = test['Credit_Product'].replace(['Yes','No'],[1,0])

In [ ]:
from sklearn.preprocessing import LabelEncoder

for c in train.columns:
    if train[c].dtype=='object': 
        lbl = LabelEncoder()
        lbl.fit(list(train[c].values) + list(test[c].values))
        train[c] = lbl.transform(train[c].values)
        test[c] = lbl.transform(test[c].values)
        
display(train.head())
display(test.head())

,ID,Gender,Age,Region_Code,Occupation,Channel_Code,Vintage,Credit_Product,Avg_Account_Balance,Is_Active,Is_Lead
0,255836,0,73,18,1,2,43,0,1045696,0,0
1,187146,0,30,27,2,0,32,0,581988,0,0
2,173604,0,56,18,3,2,26,0,1484315,1,0
3,81840,1,34,20,2,0,19,0,470454,0,0
4,299455,0,30,32,2,0,33,0,886787,0,0


,ID,Gender,Age,Region_Code,Occupation,Channel_Code,Vintage,Credit_Product,Avg_Account_Balance,Is_Active
0,314009,1,29,4,1,0,25,1,742366,0
1,96047,1,43,18,1,1,49,0,925537,0
2,316206,1,31,20,2,0,14,0,215949,0
3,303077,1,29,22,1,0,33,0,868070,0
4,292640,0,29,20,1,0,19,0,657087,0


In [ ]:
target = train.pop('Is_Lead')

In [ ]:
from xgboost import XGBClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score

aucroc_arr = []
# Making the final training
kf = StratifiedKFold(n_splits=5)
for train_index, test_index in kf.split(train,target):
    X_train, X_test = train.iloc[train_index], train.iloc[test_index]
    y_train, y_test = target.iloc[train_index],target.iloc[test_index]
    model = XGBClassifier(
        n_estimators=10000,
        verbosity=1,
        eval_metric="auc",
        tree_method="gpu_hist",
        gpu_id=0,
        alpha=7.105038963844129,
        colsample_bytree=0.25505629740052566,
        gamma=0.4999381950212869,
        reg_lambda=1.7256912198205319,
        learning_rate=0.011823142071967673,
        max_bin=338,
        max_depth=8,
        min_child_weight=2.286836198630466,
        subsample=0.618417952155855,
    )
    model.fit(X_train,y_train,eval_set=[(X_test,y_test)],early_stopping_rounds=100,verbose=False)
    preds = model.predict_proba(X_test)[:,1]
    preds1 = model.predict(X_test)
    print(preds)
    print(preds1)
    aucroc = roc_auc_score(y_test,preds)
    print(aucroc)
    aucroc_arr.append(aucroc)

[0.16453505 0.0575869  0.29564548 ... 0.03884257 0.07243134 0.06244329]
[0 0 0 ... 0 0 0]
0.7802830347706061
[0.3339862  0.3195423  0.41245472 ... 0.3536103  0.05982167 0.36059505]
[0 0 0 ... 0 0 0]
0.7811706334343023
[0.940791   0.37794557 0.22650607 ... 0.3395094  0.14966965 0.08646096]
[1 0 0 ... 0 0 0]
0.7788618736945294
[0.23629554 0.40524903 0.96363485 ... 0.2593597  0.5356718  0.08510824]
[0 0 1 ... 0 1 0]
0.7826452519437597
[0.0831686  0.20902091 0.27762094 ... 0.07687277 0.06866209 0.083587  ]
[0 0 0 ... 0 0 0]
0.7798811920035978


In [ ]:
!pip3 install catboost

     |████████████████████████████████| 67.3MB 55kB/s 


In [ ]:
from catboost import CatBoostClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score
aucroc_cb = []
kf = StratifiedKFold(n_splits=5)
for train_index, test_index in kf.split(train,target):
    X_train, X_test = train.iloc[train_index], train.iloc[test_index]
    y_train, y_test = target.iloc[train_index],target.iloc[test_index]
    clf = CatBoostClassifier(
    iterations=5)
    clf.fit(X_train,y_train,eval_set = (X_test,y_test))
    preds = clf.predict_proba(data = X_test)[:,1]
    aucroc = roc_auc_score(y_test,preds)
    print(aucroc)
    aucroc_cb.append(aucroc)


    

Learning rate set to 0.5
0:	learn: 0.5453049	test: 0.5456190	best: 0.5456190 (0)	total: 36.8ms	remaining: 147ms
1:	learn: 0.4930065	test: 0.4934023	best: 0.4934023 (1)	total: 76.8ms	remaining: 115ms
2:	learn: 0.4691011	test: 0.4694882	best: 0.4694882 (2)	total: 118ms	remaining: 78.4ms
3:	learn: 0.4573402	test: 0.4578536	best: 0.4578536 (3)	total: 156ms	remaining: 39.1ms
4:	learn: 0.4497790	test: 0.4504246	best: 0.4504246 (4)	total: 209ms	remaining: 0us

bestTest = 0.4504246452
bestIteration = 4

0.7808973310280027
Learning rate set to 0.5
0:	learn: 0.5438059	test: 0.5434808	best: 0.5434808 (0)	total: 38.7ms	remaining: 155ms
1:	learn: 0.4929224	test: 0.4927186	best: 0.4927186 (1)	total: 77.1ms	remaining: 116ms
2:	learn: 0.4710873	test: 0.4713899	best: 0.4713899 (2)	total: 118ms	remaining: 78.4ms
3:	learn: 0.4590326	test: 0.4596364	best: 0.4596364 (3)	total: 157ms	remaining: 39.3ms
4:	learn: 0.4510245	test: 0.4515154	best: 0.4515154 (4)	total: 196ms	remaining: 0us

bestTest = 0.451515426

In [ ]:
from sklearn import linear_model
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold
aucroc_lr = []
kf = StratifiedKFold(n_splits=5)
for train_index, test_index in kf.split(train,target):
    X_train, X_test = train.iloc[train_index], train.iloc[test_index]
    y_train, y_test = target.iloc[train_index],target.iloc[test_index]
    lr = linear_model.LogisticRegression()
    lr.fit(X_train,y_train)
    preds = clf.predict_proba(data = X_test)[:,1]
    aucroc = roc_auc_score(y_test,preds)
    print(aucroc)
    aucroc_cb.append(aucroc)



0.7805375624749404
0.779568700210026
0.7799646806592615
0.782563168247042
0.7795826862799485


In [ ]:
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold
import lightgbm as lgb
aucroc_lgbm = []
kf = StratifiedKFold(n_splits=10)
for train_index, test_index in kf.split(train,target):
    X_train, X_test = train.iloc[train_index], train.iloc[test_index]
    y_train, y_test = target.iloc[train_index],target.iloc[test_index]
    params_lgb={'n_jobs':-1,
            'learning_rate':0.0094,
            'n_estimators':10000,
            'colsample_bytree':0.94,
            'subsample' : 0.75,
            'subsample_freq' : 1,
            'reg_alpha': 1.0,
            'reg_lambda' : 5.0,
            'random_state':42,
            'eval_set' :(X_test,y_test),
            'eval_metric' : 'auc',
            'early_stopping_rounds' : 100,
            'verbose' : 42}
    dtrain = lgb.Dataset(X_train,y_train)
    dvalid = lgb.Dataset(X_test,y_test)
    clf1 = lgb.train(params_lgb,dtrain,eval_metric='auc')

    preds = clf.predict_proba(X_test)[:,1]
    #preds1 = clf.predict_proba(X_test)[:,1]
    #aucroc1 = roc_auc_score(y_test,preds1)
    aucroc = roc_auc_score(y_test,preds)
    print(aucroc)
    #print(aucroc1)
    aucroc_lgbm.append(aucroc)


TypeError: ignored